# MAKING THIS WORK

In [1]:
# Common Stuffs
import os, sys, time, datetime, collections

In [2]:
# Tensorflow Stuffs
import tensorflow as tf
import pandas as pd
from collections import deque
import numpy as np
import random

In [3]:
# import Finance Stuffs
import ccxt

In [5]:
from plotter import Plot
import plotter
plotter.output_notebook()

Loading BokehJS ...

In [6]:
from stock_env import StockEnv

In [7]:
from DQN_trade import DQN_Trade as bot

In [9]:
# show all exchanges that will give it up.
if False:
    for i in ccxt.exchanges:
        this_exchange = eval('ccxt.{}()'.format(i))
        if this_exchange.hasFetchOHLCV:
            print(this_exchange.id)

In [10]:
def fix_time(df):
    sLen = len(df['time'])
    df['time_str'] = pd.Series(''*sLen)
    for i, e in enumerate(df['time']):
        print(e)
        break
        if e == 0:
            df['time_str'][i] = 'NaN'
            continue
        realtime = int(int(e)/1000)
        print('is {} -> {}'.format(e, realtime))
        time = datetime.datetime.fromtimestamp(realtime) #.strftime('%Y-%m-%d %H:%M:%S')
        time_str = datetime.datetime.fromtimestamp(realtime).strftime('|%Y-%m-%d %H:%M:%S|')
        #print(time)
        #df['time'][i] = time
        df['time_str'][i] = time_str
        
    print('Changed to datetime format!')
    return df

In [11]:
# TESTING GET CHARTS FROM CCXT
if False:
    pair = 'BTC/USDT'
    for e in ccxt.exchanges:
        if 'poloniex' in e:
            this_exchange = eval('ccxt.{}()'.format(e))
            if this_exchange.hasFetchOHLCV:
                time.sleep (this_exchange.rateLimit / 1000) # time.sleep wants seconds
                print(this_exchange.id)
                # NEED 2 CHARTS 1 for low time, 1 for High Times... lol
                lowtime_chart = this_exchange.fetchOhlcv(pair, 
                                                         timeframe = '5m', 
                                                         since = int(1514764800), 
                                                         #limit = undefined, 
                                                         #params = {}
                                                         )
                df = fix_time(pd.DataFrame(lowtime_chart, columns=['time','open','high','low','close','vol']))
                print('Elements in Dataset:', len(df))
                print('Collected All Datasets')

In [12]:
pair = 'BTC/USDT'
period = '5m'
msec = 1000
minute = 5 * 60 * msec
hold = 30

for e in ccxt.exchanges:
    if 'poloniex' in e:
        config = {'rateLimit': 3000,
                  'enableRateLimit': True,
                  #'verbose': True,
                 }
        exchange = this_exchange = eval('ccxt.{}({})'.format(e, config))
        print('Searching Exchange: {} -> {}'.format(exchange.id, pair))
        from_datetime = '2018-01-15 03:00:00'
        from_timestamp = exchange.parse8601(from_datetime)
        print(from_timestamp)
        now = exchange.milliseconds()
        
        df = pd.DataFrame([(0,0,0,0,0,0)], columns=['time','open','high','low','close','vol'])
        while from_timestamp < now:
            try:
                time.sleep (this_exchange.rateLimit / 1000) # time.sleep wants seconds
                # print(exchange.milliseconds(), 'Fetching candles starting from', exchange.iso8601(from_timestamp))
                print("Start Time: {}".format(exchange.iso8601(from_timestamp)))
                
                #########
                ohlcvs = exchange.fetch_ohlcv(pair, period, from_timestamp)
                new_data = pd.DataFrame(ohlcvs, columns=['time','open','high','low','close','vol'])
                print('Caught {} pokemons'.format(len(ohlcvs)))
                timestamp_first_row = int(new_data.loc[0]['time'])
                timestamp_last_row = int(new_data.loc[len(ohlcvs)-1]['time'])
                print('timestamp first row: {} is {}'.format(timestamp_first_row, exchange.iso8601(timestamp_first_row)))
                print('timestamp last row: {} is {}'.format(timestamp_last_row, exchange.iso8601(timestamp_last_row)))
                from_timestamp = int(int(timestamp_last_row) + minute)
                if len(new_data) > 0:
                    df = df.append(new_data)
                
            except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:
                print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
                time.sleep(hold)
        # AFTER COLLECTION BURN THE FIRST INDEX
        df = df[1:]
        df = fix_time(df)
        # ... OTHER STUFFS
        
        #plot it
        if True:
            Plot.Plot_me(df)
            
        # print it
        if True:
            print(df.loc[0])
            print(len(df))
            
        # Save it
        if True:
            df = df.drop(['time_str'], axis=1)
            df.to_csv('./alphagriffin.csv', index=False, header=False)
            print('Saved!')

Searching Exchange: poloniex -> BTC/USDT
1515985200000
Start Time: 2018-01-15T03:00:00.000Z
Caught 1802 pokemons
timestamp first row: 1515985200000 is 2018-01-15T03:00:00.000Z
timestamp last row: 1516525500000 is 2018-01-21T09:05:00.000Z
1515985200000
Changed to datetime format!


time        1515985200000
open                13463
high                13463
low                 13380
close               13380
vol               95348.6
time_str                 
Name: 0, dtype: object
1802
Saved!


In [13]:
# print(df)

               time          open          high           low         close  \
0     1515985200000  13462.979708  13462.979708  13380.000000  13380.000000   
1     1515985500000  13380.000000  13381.002446  13320.000000  13369.003186   
2     1515985800000  13369.003185  13375.000000  13304.025963  13375.000000   
3     1515986100000  13375.000000  13388.084057  13361.693628  13388.084057   
4     1515986400000  13388.084057  13416.000000  13388.084057  13388.084058   
5     1515986700000  13412.003178  13412.003178  13369.000686  13369.000686   
6     1515987000000  13395.000686  13395.000686  13359.508118  13375.003186   
7     1515987300000  13390.582019  13390.582019  13364.009509  13390.582019   
8     1515987600000  13390.000000  13443.568015  13390.000000  13436.689232   
9     1515987900000  13436.689232  13474.098765  13408.009507  13473.500000   
10    1515988200000  13444.741705  13528.000000  13444.741705  13524.009506   
11    1515988500000  13528.000000  13528.000000  134

In [14]:
# save dataframe to csv file
cols = []
for i in range(20):
    cols.append('close_{}'.format(i))
    cols.append('vol_{}'.format(i))

working_df  = pd.DataFrame(columns=cols)
for index in range(len(df)):
    # start once we have 20 samples!
    if index <= len(df) - 20:
        
        # we are using 20 data samples per line
        dataline = []
        for i in range(20):
            time = df.loc[i+index]['time']
            close = df.loc[i+index]['close']
            vol = df.loc[i+index]['vol']
            #print('{} | {} | close: {}, vol: {}'.format(i, exchange.iso8601(time), close, vol))
            dataline.append(float(close))
            dataline.append(float(vol))
        new_df = pd.DataFrame([dataline], columns=cols)
        working_df = working_df.append(new_df)

In [15]:
if False:
    print(working_df)

         close_0         vol_0       close_1         vol_1       close_2  \
0   13380.000000  9.534856e+04  13369.003186  1.250866e+05  13375.000000   
0   13369.003186  1.250866e+05  13375.000000  6.001553e+05  13388.084057   
0   13375.000000  6.001553e+05  13388.084057  2.833012e+04  13388.084058   
0   13388.084057  2.833012e+04  13388.084058  3.859752e+04  13369.000686   
0   13388.084058  3.859752e+04  13369.000686  8.420472e+04  13375.003186   
0   13369.000686  8.420472e+04  13375.003186  2.887923e+04  13390.582019   
0   13375.003186  2.887923e+04  13390.582019  4.750388e+04  13436.689232   
0   13390.582019  4.750388e+04  13436.689232  8.989872e+04  13473.500000   
0   13436.689232  8.989872e+04  13473.500000  2.110607e+04  13524.009506   
0   13473.500000  2.110607e+04  13524.009506  4.834404e+04  13469.249420   
0   13524.009506  4.834404e+04  13469.249420  8.966047e+04  13500.381084   
0   13469.249420  8.966047e+04  13500.381084  8.735545e+04  13547.123137   
0   13500.38

In [16]:
working_np = working_df.as_matrix()

In [17]:
if False:
    print(working_np)

[[  13380.           95348.5596967    13369.00318575 ...,   44926.43478023
    13570.          141168.01602521]
 [  13369.00318575  125086.62391337   13375.         ...,  141168.01602521
    13579.875176    152070.79387687]
 [  13375.          600155.2980039    13388.0840573  ...,  152070.79387687
    13615.           72415.49557171]
 ..., 
 [  11831.00000008  781509.47106424   11831.10000003 ...,   96166.58878606
    11850.50354047  233280.41433312]
 [  11831.10000003  468550.72221576   11770.00000024 ...,  233280.41433312
    11745.00000009  236971.70756649]
 [  11770.00000024  476334.80201756   11711.7        ...,  236971.70756649
    11701.00000221   73711.54635351]]


In [18]:
# angent = DQN_Trade()  # edit
Agent = bot()

INFO:tensorflow:Restoring parameters from saved_network\network-dqn-7
load model  success


In [19]:
# AFTER DATA IS BUILD
# data = np.loadtxt('./alphagriffin.csv', delimiter = ',', skiprows=1)
# data = data[230:-1]  #delete the first day data
data = working_np

In [20]:
if False:
    print(data)

In [21]:
if True:
    for i in range(0, 10):
        print("Round: {}".format(i))
        iters = int(len(data)/240)
        print("iters: {}/round".format(iters))
        for iter_step in range(0, iters):
            print("Iter step {}".format(iter_step))
            iter_data = data[
                iter_step*240 : iter_step*240+240
            ]
            # build data set and reset to 0
            env = StockEnv(iter_data)
            s = env.reset()
            
            # until break
            while True:
                # first S is data 0 from first dataloop
                # s is recycled from bottom of loop
                action = Agent.egreedy_action(s)
                
                s_, reward, done = env.gostep(action)
                #print ("Action: {} | 0 = buy, 1 = sell".format(action))
                #print ("Reward: {:.4f}".format(reward))
                # print ("Current Price set: {}".format(s_))
                Agent.precive(s,action,reward,s_,done)
                #print ("Prediction made...")
                #print("Total Cash on hand?: {}".format(env.cash))
                s= s_
                if done:
                    # print("done")
                    print("Total Cash on hand?: {}".format(env.cash))
                    break
            #break
        #break
        Agent.save_model(step=i)

Round: 0
iters: 7/round
Iter step 0
Total Cash on hand?: 302459.6723467899
Iter step 1
Total Cash on hand?: 522958.1293064902
Iter step 2
Total Cash on hand?: -246912.6553284601
Iter step 3
Total Cash on hand?: -303682.94094641996
Iter step 4
Total Cash on hand?: -485788.5883701502
Iter step 5
Total Cash on hand?: -377115.56317589997
Iter step 6
Total Cash on hand?: -252538.40262783002
Round: 1
iters: 7/round
Iter step 0
Total Cash on hand?: -127292.71221455996
Iter step 1
Total Cash on hand?: 290362.1827925399
Iter step 2
Total Cash on hand?: -191600.86033087005
Iter step 3
Total Cash on hand?: -220329.70876755004
Iter step 4
Total Cash on hand?: -192999.75066284012
Iter step 5
Total Cash on hand?: 18002.677830660035
Iter step 6
Total Cash on hand?: -339541.44825193
Round: 2
iters: 7/round
Iter step 0


KeyboardInterrupt: 